In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\Lung-Infection\\notebooks'

In [2]:
import os 


os.chdir('../')
%pwd

'c:\\Users\\44787\\Desktop\\Lung-Infection'

In [3]:
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib


import warnings
warnings.filterwarnings("ignore")

In [64]:
mlflow.set_tracking_uri('http://localhost:2024')

runs = mlflow.search_runs(experiment_ids=[2])

runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.validation_loss,metrics.accuracy,metrics.validation_accuracy,metrics.val_loss,...,params.validation_split,params.opt_learning_rate,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.parentRunId,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.source.git.commit,tags.mlflow.latest_checkpoint_artifact,tags.mlflow.log-model.history
0,14bd058de4e04cb3a0175e3b67366ff0,2,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-07-01 14:45:07.659000+00:00,2024-07-01 14:45:07.719000+00:00,NaN,NaN,NaN,NaN,...,None,None,LOCAL,44787,a2b87cd9a40f413b8d401d576eb74ec8,capable-lark-267,.\src\pipeline\stage_02_training_and_evaluatio...,5a90075f8fbec68e09a7dd4db29ace756d9f0f12,None,None
1,e73ec21a685a44b3a071c88005af7355,2,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-07-01 14:45:07.522000+00:00,2024-07-01 14:45:07.608000+00:00,NaN,NaN,NaN,NaN,...,None,None,LOCAL,44787,a2b87cd9a40f413b8d401d576eb74ec8,colorful-shrike-505,.\src\pipeline\stage_02_training_and_evaluatio...,5a90075f8fbec68e09a7dd4db29ace756d9f0f12,None,None
2,a2b87cd9a40f413b8d401d576eb74ec8,2,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-07-01 14:43:37.610000+00:00,2024-07-01 14:45:25.841000+00:00,0.469061,0.7250,0.800,0.469061,...,0.0,0.0010000000474974513,LOCAL,44787,None,Transfer_learning_Model,.\src\pipeline\stage_02_training_and_evaluatio...,5a90075f8fbec68e09a7dd4db29ace756d9f0f12,checkpoints/latest_checkpoint.h5,"[{""run_id"": ""a2b87cd9a40f413b8d401d576eb74ec8""..."
3,1d4d94173dfa45108a44209fc6630529,2,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-06-21 21:28:00.901000+00:00,2024-06-21 21:29:17.095000+00:00,0.488761,0.7450,0.775,0.488761,...,0.0,0.0010000000474974513,LOCAL,44787,None,Transfer_learning_Model,.\demo.py,6761943fc66f4f52acf0bed72f7b3dd87d82f716,checkpoints/latest_checkpoint.h5,"[{""run_id"": ""1d4d94173dfa45108a44209fc6630529""..."
4,fc16f462890b4c1fbdbf54389db888ed,2,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-06-17 21:19:26.138000+00:00,2024-06-17 21:21:14.578000+00:00,0.504978,0.7425,0.760,0.504978,...,0.0,0.0010000000474974513,LOCAL,44787,None,Transfer_learning_Model,.\src\pipeline\stage_02_training_and_evaluatio...,2f116376a67f179c0565e3dadf9f8fa26152bfda,checkpoints/latest_checkpoint.h5,"[{""run_id"": ""fc16f462890b4c1fbdbf54389db888ed""..."
5,7d85fb424e174c4c97145a33fc4a5f52,2,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-06-16 22:08:52.813000+00:00,2024-06-16 22:11:22.642000+00:00,0.492686,0.7450,0.775,0.492686,...,0.0,0.0010000000474974513,LOCAL,44787,None,Transfer_learning_Model,src\pipeline\stage_02_training_and_evaluation.py,8872ff046b51a522835e241f9641ba4d9b608158,checkpoints/latest_checkpoint.h5,"[{""run_id"": ""7d85fb424e174c4c97145a33fc4a5f52""..."


In [65]:
# delete the run
# mlflow.delete_run('540b1b03510948f8a3918f9d5517c18c')

In [66]:
runs['metrics.validation_accuracy'].sort_values(ascending=False, ignore_index=True)

0    0.800
1    0.775
2    0.775
3    0.760
4      NaN
5      NaN
Name: metrics.validation_accuracy, dtype: float64

In [67]:
highest = runs["metrics.validation_accuracy"].sort_values(ascending=False, ignore_index=True)[0]
highest

0.800000011920929

In [68]:
runs[runs["metrics.validation_accuracy"]==highest]

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.validation_loss,metrics.accuracy,metrics.validation_accuracy,metrics.val_loss,...,params.validation_split,params.opt_learning_rate,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.parentRunId,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.source.git.commit,tags.mlflow.latest_checkpoint_artifact,tags.mlflow.log-model.history
2,a2b87cd9a40f413b8d401d576eb74ec8,2,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-07-01 14:43:37.610000+00:00,2024-07-01 14:45:25.841000+00:00,0.469061,0.725,0.8,0.469061,...,0.0,0.0010000000474974513,LOCAL,44787,None,Transfer_learning_Model,.\src\pipeline\stage_02_training_and_evaluatio...,5a90075f8fbec68e09a7dd4db29ace756d9f0f12,checkpoints/latest_checkpoint.h5,"[{""run_id"": ""a2b87cd9a40f413b8d401d576eb74ec8""..."


In [69]:
runs[runs["metrics.validation_accuracy"]==highest]['run_id']

2    a2b87cd9a40f413b8d401d576eb74ec8
Name: run_id, dtype: object

In [70]:
highest_run_id = runs[runs["metrics.validation_accuracy"]==highest]['run_id']
highest_run_id = highest_run_id.reset_index(drop=True)[0]
highest_run_id

'a2b87cd9a40f413b8d401d576eb74ec8'

In [71]:
model_name = 'EfficientNet'

client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)
    # print(mv)

    if mv["run_id"] == highest_run_id:
        current_version = mv["version"]
        logged_model = mv["source"]
        pprint(mv, indent=4)

{   'aliases': [],
    'creation_timestamp': 1719845125810,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1719845125810,
    'name': 'EfficientNet',
    'run_id': 'a2b87cd9a40f413b8d401d576eb74ec8',
    'run_link': '',
    'source': 'file:///C:/Users/44787/Desktop/Lung-Infection/mlflow_artifacts/2/a2b87cd9a40f413b8d401d576eb74ec8/artifacts/model.h5',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '4'}


In [72]:
model_name = 'EfficientNet'

client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)

    if mv["run_id"] == highest_run_id:
        current_version = mv["version"]
        logged_model = mv["source"]
        run_id = mv["run_id"]
        pprint(mv, indent=4)
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Production"
        )
    else:
        current_version = mv["version"]
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Staging"
        )  

{   'aliases': [],
    'creation_timestamp': 1719845125810,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1719845125810,
    'name': 'EfficientNet',
    'run_id': 'a2b87cd9a40f413b8d401d576eb74ec8',
    'run_link': '',
    'source': 'file:///C:/Users/44787/Desktop/Lung-Infection/mlflow_artifacts/2/a2b87cd9a40f413b8d401d576eb74ec8/artifacts/model.h5',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '4'}


In [29]:
from pathlib import Path

In [34]:
import tensorflow as tf

In [73]:
model_uri = f"runs:/{run_id}/model.h5"
loaded_model = mlflow.tensorflow.load_model(model_uri)

save_path = "saved_model.h5"
loaded_model.save(save_path)

TypeError: cannot pickle 'module' object

In [74]:
model = mlflow.tensorflow.load_model(logged_model)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling          │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        81,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,217,767 (16.09 MB)

 Trainable params: 84,097 (328.50 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 84,099 (328.52 KB)

In [51]:
loaded_model = mlflow.pyfunc.load_model(logged_model)

model_path = Path("artifacts\LogProductionModel\mlflow_model.h5")
joblib.dump(loaded_model, model_path)


['artifacts\\LogProductionModel\\mlflow_model.h5']